<a href="https://colab.research.google.com/github/and-rgr/contradiction_and_entailment/blob/main/contradiction_and_entailment_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Based on the following Kaggle notebook:
# https://github.com/sukanyabag/Detecting-Contradictions-and-Entailment-in-Multilingual-Text/tree/main/Detecting%20Contradictions%20in%20Multilingual%20Text

In [ ]:
def history_log(history, filename):
    accuracy = history.history['accuracy']
    loss = history.history['loss']
    val_accuracy = history.history['val_accuracy']
    val_loss = history.history['val_loss']

    my_list = ['loss, accuracy, val_loss, val_accuracy']
    for i in range(len(accuracy)):
        line = str(round(loss[i],4)) + ", " + str(round(accuracy[i],4)) + ", " + str(round(val_loss[i],4)) + ", " + str(round(val_accuracy[i],4))
        my_list.append(line)

    with open(filename + ".csv", 'w') as f:
        for item in my_list:
            f.write("%s\n" % item)

In [ ]:
def hyperparameter_log(filename):
    with open(filename + ".txt", 'w') as f:
        f.write("model_name = " + str(model_name) + "\n")   
        f.write("tokenizer_length = " + str(tokenizer_length) + "\n")       
        f.write("learning_rate = " + str(learning_rate) + "\n")
        f.write("batch_factor = " + str(batch_factor) + "\n")
        f.write("validation_split = " + str(validation_split) + "\n")
        f.write("kernel_initializer = " + str(kernel_initializer) + "\n")
        f.write("epochs = " + str(epochs) + "\n")
        f.write("patience = " + str(patience) + "\n")
        f.write("l1 regularization = " + str(l1) + "\n")
        f.write("l2 regularization = " + str(l2) + "\n")

In [1]:
def gpu_info():
    gpu_info = !nvidia-smi
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
        print('Not connected to a GPU')
    else:
        print(gpu_info)

In [2]:
gpu_info()

Wed Mar 30 13:41:07 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers

In [ ]:
# required by "joeddav/xlm-roberta-large-xnli", "microsoft/Multilingual-MiniLM-L12-H384"
# may need a kernel restart
!pip install sentencepiece
import sentencepiece

In [ ]:
import datetime
import pandas as pd
import tensorflow as tf
from tensorflow.keras import regularizers

In [ ]:
pd.set_option('display.expand_frame_repr', False)

In [ ]:
# set up the TPU
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    strategy = tf.distribute.get_strategy() # for CPU and single GPU

print('Number of replicas:', strategy.num_replicas_in_sync)

Number of replicas: 1


In [ ]:
### LOAD DATA ###

In [ ]:
dataset = pd.read_csv('dataset.csv')
train_labels = dataset['label']
train = dataset.drop(columns='label')

In [ ]:
print("training data shape: \t", train.shape)
print("training labels shape: \t", train_labels.shape)

training data shape: 	 (12120, 5)
training labels shape: 	 (12120,)


In [ ]:
### SET HYPERPARAMETERS ###

In [ ]:
# why is the first epoch almost always the best??
# best accuracy: 0.9043
model_name = "joeddav/xlm-roberta-large-xnli"

# OOM with "joeddav/xlm-roberta-large-xnli" and with values larger than ~160
tokenizer_length = 150

learning_rate =  0.00002
batch_factor = 16
validation_split = 0.3
kernel_initializer = "lecun_normal"
epochs = 5
patience = 10
l1 = 0.003
l2 = 0.003

In [ ]:
### SELECT MODEL ###

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

# models
# https://huggingface.co/models?library=tf&sort=downloads

# https://huggingface.co/microsoft/Multilingual-MiniLM-L12-H384
# model_name = "microsoft/Multilingual-MiniLM-L12-H384"
# best accuracy: 0.6782

# *
# https://huggingface.co/bert-base-multilingual-uncased?text=Paris+is+the+%5BMASK%5D+of+France.
# model_name = "bert-base-multilingual-uncased"
# best accuracy: 0.6155

# **
# https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment
# model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
# best accuracy: 0.6097

# https://huggingface.co/distilbert-base-multilingual-cased
# model_name = "distilbert-base-multilingual-cased"
# best accuracy: 0.5974

# **
# https://huggingface.co/amberoad/bert-multilingual-passage-reranking-msmarco
# model_name = "amberoad/bert-multilingual-passage-reranking-msmarco"
# best accuracy: 0.5941

# model_name = "distilbert-base-uncased-finetuned-sst-2-english"
# best accuracy: 0.4942

# model_name = "bert-base-multilingual-cased"
# best accuracy: 0.3828

# model_name = "clips/mfaq"
# best accuracy: 0.3094

# ***
# https://huggingface.co/sentence-transformers/multi-qa-MiniLM-L6-cos-v1
# model_name = "sentence-transformers/multi-qa-MiniLM-L6-cos-v1"
# sentence-transformers/multi-qa-MiniLM-L6-cos-v1 does not appear to have a file named tf_model.h5 but there is a file for PyTorch weights. Use `from_pt=True` to load this model from those weights.


In [ ]:
### TOKENIZATION ###

In [ ]:
# premise average length: 107
# hypothesis average length: 54
print('min length of \"premise\" column:', train['premise'].apply(len).min())
print('average length of \"premise\" column:', round(train['premise'].apply(len).mean()))
print('max length of \"premise\" column:', train['premise'].apply(len).max())

print('min length of \"hypothesis\" column:', train['hypothesis'].apply(len).min())
print('average length of \"hypothesis\" column:', round(train['hypothesis'].apply(len).mean()))
print('max length of \"hypothesis\" column:', train['hypothesis'].apply(len).max())

min length of "premise" column: 4
average length of "premise" column: 107
max length of "premise" column: 967
min length of "hypothesis" column: 4
average length of "hypothesis" column: 54
max length of "hypothesis" column: 276


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

train_batch = tokenizer.batch_encode_plus(
    train[['premise','hypothesis']].values.tolist(),
    padding='max_length',
    truncation=True,
    max_length=tokenizer_length,
    return_attention_mask=True,
)

In [ ]:
train_tf1=tf.convert_to_tensor(train_batch['input_ids'],dtype=tf.int32)
train_tf2=tf.convert_to_tensor(train_batch['attention_mask'],dtype=tf.int32)
train_input={'input_word_ids':train_tf1,'input_mask':train_tf2}

In [ ]:
### TRAIN MODEL ###

In [ ]:
# TO DO
# cross validation
    # https://towardsdatascience.com/cross-validation-in-machine-learning-72924a69872f
    # https://machinelearningmastery.com/k-fold-cross-validation/
    # https://github.com/christianversloot/machine-learning-articles/blob/main/how-to-use-k-fold-cross-validation-with-keras.md
# data augmentation for nlp

# solution to test accuracy decreasing over time: same as overfitting

# https://www.tensorflow.org/tutorials/keras/keras_tuner
# hyperparameter search between l1 and l2 regularization values \in (0.01,0.025)
# dropout values \in (0.01, 0.1)

# try l1 regularization = 0.01, l2 regularization = 0.01
# if globalaveragepooling helps fix overfitting, must explain WHY

# maybe add:
#   learning rate decay
#   checkpoints

# BERT is trained to receive data as tensors, pairs of sentences distinguished by lists of 1's and 0's, and separated by the CLS and SEP tokens

In [ ]:
# OOM TEST

# alpha = [0.01, 0.02, 0.025, 0.03, 0.05]
# alpha = [0.01, 0.05]

# for l1 in alpha:
#     for l2 in alpha:
#         # works fine outside the loop. OOM with loop, even with [:200] data size
# OOM with loop happens when reloading the model 


with strategy.scope():
    input_word_ids = tf.keras.Input(shape=(tokenizer_length,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(tokenizer_length,), dtype=tf.int32, name="input_mask")
    tf_model = TFAutoModelForSequenceClassification.from_pretrained(model_name)
    embedding = tf_model([input_word_ids, input_mask])[0]
    # embedding = tf.keras.layers.GlobalAveragePooling1D()(embedding)
    # Input 0 of layer "global_average_pooling1d" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 3)
    # https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense    
    output = tf.keras.layers.Dense(3, 
                                kernel_initializer = kernel_initializer,  
                                kernel_regularizer = regularizers.l1_l2(l1 = l1, l2 = l2),          
                                activation = 'softmax')(embedding)
    # output = tf.keras.layers.Dropout(dropout)(output)
    model = tf.keras.Model(inputs = [input_word_ids,input_mask], outputs = output)
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate),
                loss = 'sparse_categorical_crossentropy',
                metrics = ['accuracy'])
    # model.summary()

timestamp = datetime.datetime.now().strftime('%Y-%m-%d__%H-%M-%S')

early_stop = tf.keras.callbacks.EarlyStopping(patience = patience, restore_best_weights = True, verbose = 1, monitor = 'val_accuracy')
checkpoint = tf.keras.callbacks.ModelCheckpoint(timestamp + ".hdf5", verbose = 0, save_best_only = True, monitor = 'val_accuracy')

# a batch size of 8 requires about 15GB of GPU memory
history = model.fit(train_input, train_labels, epochs = epochs, verbose = 1, 
        batch_size = batch_factor * strategy.num_replicas_in_sync, validation_split = validation_split, 
        shuffle = True, callbacks=[early_stop, checkpoint])

history_log(history, "training_log_" + timestamp)
hyperparameter_log("hyperparameter_log_" + timestamp)

All model checkpoint layers were used when initializing TFXLMRobertaForSequenceClassification.

All the layers of TFXLMRobertaForSequenceClassification were initialized from the model checkpoint at joeddav/xlm-roberta-large-xnli.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaForSequenceClassification for predictions without further training.


Epoch 1/3
2121/2121 [==============================] - 835s 382ms/step - loss: 0.6281 - accuracy: 0.7674 - val_loss: 0.5059 - val_accuracy: 0.8146
Epoch 2/3
 356/2121 [====>.........................] - ETA: 9:31 - loss: 0.4461 - accuracy: 0.8483

KeyboardInterrupt: ignored

In [ ]:
gpu_info()

In [ ]:
# early_stop = tf.keras.callbacks.EarlyStopping(patience = patience, restore_best_weights = True, verbose = 1, monitor = 'val_accuracy')

# # filename = "dropout__" + str(dropout) + "__l1__" + str(l1) + "__l2_" + str(l2)
# timestamp = datetime.datetime.now().strftime('%Y-%m-%d__%H-%M-%S')

# checkpoint = tf.keras.callbacks.ModelCheckpoint(timestamp + ".hdf5", verbose = 0, save_best_only = True, monitor = 'val_accuracy')

# history = model.fit(train_input, train_labels, epochs = epochs, verbose = 1, 
#           batch_size = batch_factor * strategy.num_replicas_in_sync, validation_split = validation_split, 
#           shuffle = True, callbacks=[early_stop, checkpoint])

# # save training history and hyperparameters to log file
# history_log(history, "training_log_" + timestamp + ".txt")
# hyperparameter_log("hyperparameter_log_" + timestamp + ".txt")

# # how to store the hyperparameters?
# # how to store the hyperparameters, model, and training history all together? pickle file?

In [ ]:
# # https://www.tensorflow.org/api_docs/python/tf/keras/Model
# # history = model.fit(train_input, train_labels, epochs = 10, verbose = 1, 

# # a batch size of 8 requires about 15GB of GPU memory
# model.fit(train_input, train_labels, epochs = epochs, verbose = 1, 
#           batch_size = batch_factor * strategy.num_replicas_in_sync, validation_split = validation_split, 
#           shuffle = True, callbacks=[early_stop])


In [ ]:
8# # write training history to file

# with open("Output.txt", "w") as text_file:
#     line = str(history.history)
#     text_file.write(line)

# with open("History.txt", 'wb') as file_pi:
#     pickle.dump(history.history, file_pi)

In [ ]:
# # compare predictions and test_labels
# predictions = [np.argmax(i) for i in model.predict(test_input)]
# accuracy_list = [1 if predictions[i] == list(test_labels)[i] else 0 for i in range(len(test_labels))]
# accuracy = accuracy_list.count(1)/len(test_labels)
# print(round(accuracy,4))